In [53]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


In [10]:
path = 'California_Fire_Incidents.xlsx'
fire = pd.read_excel(path)
columns = ['AcresBurned', 'AirTankers', 'AdminUnit', 'Counties', 'CountyIds', 'Engines', 'Latitude', 'Longitude', 'Started']

In [11]:
fire_df = fire[columns]
fire_df

,AcresBurned,AirTankers,AdminUnit,Counties,CountyIds,Engines,Latitude,Longitude,Started
0,257314.0,NaN,Stanislaus National Forest/Yosemite National Park,Tuolumne,55,NaN,37.857000,-120.086000,2013-08-17T15:25:00Z
1,30274.0,NaN,USFS Angeles National Forest/Los Angeles Count...,Los Angeles,19,NaN,34.585595,-118.423176,2013-05-30T15:28:00Z
2,27531.0,NaN,CAL FIRE Riverside Unit / San Bernardino Natio...,Riverside,33,NaN,33.709500,-116.728850,2013-07-15T13:43:00Z
3,27440.0,NaN,Tahoe National Forest,Placer,31,NaN,39.120000,-120.650000,2013-08-10T16:30:00Z
4,24251.0,NaN,Ventura County Fire/CAL FIRE,Ventura,56,117.0,0.000000,0.000000,2013-05-02T07:01:00Z
...,...,...,...,...,...,...,...,...,...
1631,9.0,NaN,CAL FIRE / Riverside County Fire,Riverside,33,NaN,33.827979,-117.499619,2019-10-10T12:08:00Z
1632,2.0,NaN,CAL FIRE Nevada-Yuba-Placer Unit,Nevada,29,NaN,39.409722,-121.000556,2019-06-28T15:03:04Z
1633,NaN,NaN,Yolo County Fire Protection District,Yolo,57,NaN,38.734634,-121.729691,2019-11-25T12:02:02Z
1634,NaN,NaN,Camp Pendleton Marine Corps Base,San Diego,37,NaN,33.351145,-117.403719,2019-10-22T19:20:44Z


In [12]:
fire_df.isna().sum()

AcresBurned       3
AirTankers     1608
AdminUnit         0
Counties          0
CountyIds         0
Engines        1445
Latitude          0
Longitude         0
Started           0
dtype: int64

In [13]:
fire_df = fire_df.drop(['AirTankers', 'Engines', 'AdminUnit'], axis=1)
fire_df = fire_df.dropna()

In [6]:
fire_df.tail()

,AcresBurned,Counties,CountyIds,Latitude,Longitude,Started
1628,10.0,Butte,4,39.423833,-121.579738,2019-07-23T14:41:00Z
1629,10.0,Siskiyou,47,41.946220,-122.401570,2019-06-16T20:33:00Z
1630,10.0,Butte,4,39.839580,-121.957000,2019-04-30T12:20:00Z
1631,9.0,Riverside,33,33.827979,-117.499619,2019-10-10T12:08:00Z
1632,2.0,Nevada,29,39.409722,-121.000556,2019-06-28T15:03:04Z


In [7]:
fire_df.dtypes

AcresBurned    float64
Counties        object
CountyIds       object
Latitude       float64
Longitude      float64
Started         object
dtype: object

In [20]:
months = []
years = []
for index, row in fire_df.iterrows():
   #print(date)
    month = row['Started'].split('-')[1]
    year = row['Started'].split('-')[0]
    months.append(np.float32(month))
    years.append(np.float32(year))

In [21]:
fire_df = fire_df.drop('Started', axis=1)
fire_df['Month'] = months
fire_df['Year'] = years
fire_df.head()
#fire_df.to_csv('Wildfires 2013-2019.csv')

,AcresBurned,Counties,CountyIds,Latitude,Longitude,Month,Year
0,257314.0,Tuolumne,55,37.857000,-120.086000,8.0,2013.0
1,30274.0,Los Angeles,19,34.585595,-118.423176,5.0,2013.0
2,27531.0,Riverside,33,33.709500,-116.728850,7.0,2013.0
3,27440.0,Placer,31,39.120000,-120.650000,8.0,2013.0
4,24251.0,Ventura,56,0.000000,0.000000,5.0,2013.0


In [22]:
labels = fire_df['Counties']
features = fire_df.drop(['Counties', 'CountyIds'], axis=1)


In [48]:
dummies = pd.get_dummies(labels)
one_hot_labels = dummies.to_numpy()
scaler = MinMaxScaler()
#acres = features['AcresBurned'].to_numpy().reshape(-1,1)
#scaled = scaler.fit_transform(features)
one_hot_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8)

In [24]:
#features.to_csv('cleaned_firedata.csv')

In [25]:
print(features.shape)
one_hot_labels.shape

(1633, 5)


(1633, 59)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(scaled, one_hot_labels, test_size=0.2)

In [55]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
features_scaled = normalizer(train_features)

## Rough draft model


In [56]:
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras import layers, Sequential, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, concatenate
# from tensorflow.keras.layers.core import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing
# from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input


def build_dnn():

    
    normalizer = layers.BatchNormalization(momentum=0.98, epsilon=.001, trainable=True)

    model = Sequential([

        #normalizer,
        layers.Dense(24, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    model = Model([img_input, num_input], out)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0001), metrics=['accuracy'])
    print("Model compiled")
    return model

In [57]:
def build_dnn(input_shape, numclasses):
    feature_input = Input(shape=input_shape)
    model = Dense(8, activation='relu')(feature_input)
    model = Dense(4, activation='relu')(model)
    model = Dense(1, activation='relu')(model)
    output = Dense(numclasses, activation='sigmoid')(model)
    model = Model(inputs=feature_input, outputs=output)
    print(model.summary())
    return(model)

In [58]:
model = build_dnn((5,), len(one_hot_labels[0]))

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 48        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 5         
_________________________________________________________________
dense_15 (Dense)             (None, 59)                118       
Total params: 207
Trainable params: 207
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0001), metrics=['accuracy'])

model.fit(X_train, y_train, validation_split=0.2, epochs=10)
model.evaluate(X_test, y_test, verbose=0)


Epoch 1/10
33/33 [==============================] - 0s 9ms/step - loss: 4.0772 - accuracy: 0.0747 - val_loss: 4.0768 - val_accuracy: 0.1336
Epoch 2/10
33/33 [==============================] - 0s 4ms/step - loss: 4.0765 - accuracy: 0.0872 - val_loss: 4.0762 - val_accuracy: 0.1336
Epoch 3/10
33/33 [==============================] - 0s 3ms/step - loss: 4.0757 - accuracy: 0.0872 - val_loss: 4.0755 - val_accuracy: 0.1336
Epoch 4/10
33/33 [==============================] - 0s 4ms/step - loss: 4.0750 - accuracy: 0.0872 - val_loss: 4.0748 - val_accuracy: 0.1336
Epoch 5/10
33/33 [==============================] - 0s 4ms/step - loss: 4.0743 - accuracy: 0.0872 - val_loss: 4.0741 - val_accuracy: 0.1336
Epoch 6/10
33/33 [==============================] - 0s 4ms/step - loss: 4.0735 - accuracy: 0.0872 - val_loss: 4.0735 - val_accuracy: 0.1336
Epoch 7/10
33/33 [==============================] - 0s 3ms/step - loss: 4.0728 - accuracy: 0.0872 - val_loss: 4.0728 - val_accuracy: 0.1336
Epoch 8/10
33/33 [==

[4.071535587310791, 0.05810397490859032]

In [60]:
truth = y_test.argmax(axis=1)
predic = model.predict(X_test)
print(truth)
predic = predic.argmax(axis=1)
print(predic)

[31 16  9 17 10  3 19 36 57 20  8 32 32 26 43 15 56 58 45 45 56 15 17  3
  8 45 17 26 53 41 29 36 53  9 58 32 41  6 38 56 41 45 34 54 43 28  9 27
 35 15  7 52 56 45 32 35 45  9  4 52 16 16 11 26 11 46 26  6  9 40  9 43
 48 30  4 38 22 38  3 20 36 30  6 47 52  0  2 38 19 13 20  3 56 36 35 52
 40 24 36  0 20 16  3 15 16 38 21 48 54 41 13 30 28  9 13 17 36 18  0 27
  6  3  9 13 55 37 22 11 18  8 24 14 29 38 24 36 33 26 54 22 54 55 41 28
 16 45 42  9  8 37 30 26  4  8  7 22 43 36  0 40 25 36 36 17 13 27  7 34
 13 35 38 45 28 38 18 30 15 36  6 25 15 32 47 19 55 15 34 42 32  0 20 45
 33 32  3 17 27 38 52 56 26 20  0  9 32 32 54 15 58 13 14 34 35  8 32 48
 38 52 15 26 29 21 52 20  6 56  4 43 32 16 46 53 38 13 53 32 32 36  3 46
  8  2 45 36  8 43 35 15 52  8 48 54 10 27 35 34 36 26 20 32 56 32 54  0
  3 32 16 16  4 53  3 41 38 12  4 21 36 32 36 54 36 41 53 16 24 54 17 32
 40 35  8 45 52 25 45 38  9 13 54 38 36 36 58 32  6 45 41 54 47 36 40 26
 21 36 35  3  4 52 18  6  8 54 50 20 28  6  9]
[32 

In [46]:
def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [50]:
undummify(predic)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'